<a href="https://colab.research.google.com/github/rsjain1978/advanced-nlp/blob/master/similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytorch-transformers

In [2]:
import torch
from pytorch_transformers import *
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
% matplotlib inline

from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics.pairwise import cosine_similarity


Using TensorFlow backend.


In [0]:
# CSX_0000277948_10K_20171231_Item7_excerpt
source  = ["baba baba black sheep"]
target  = ["johnny johhny yes papa"]

In [0]:
def convert_txt_to_tensor(sentence, tokenizer):
  
  MAX_LEN = 512
  
  tokenized_texts = [tokenizer.tokenize(sent) for sent in sentence]

  # Pad our input tokens
  input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

  # Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

  # Pad to 512 length
  input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

  # Convert to torch tensor
  train_inputs = torch.tensor(input_ids)
  
  return train_inputs

def get_sentence_embedding(sentence, tokenizer, model):
  # Convert sentence to tensor
  train_inputs = convert_txt_to_tensor(sentence, tokenizer)
  
  # get vector representation of all layers
  encoded_layers, all_attentions = model(train_inputs)[-2:]
  
  # sentence vector as mean of last two layers
  sentence_embedding = torch.mean(encoded_layers[11], 1)
  
  # convert to numpy array
  sentence_embedding = sentence_embedding.detach().numpy()
  
  return sentence_embedding

def find_difference (s1, s2, tokenizer, model):
  se_1 = get_sentence_embedding(s1, tokenizer, model)
  se_2 = get_sentence_embedding(s2, tokenizer, model)
  
  difference = cosine_similarity(se_1, se_2)
  return difference[0]

In [0]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
bert_model = BertModel.from_pretrained('bert-base-uncased',
                                    output_hidden_states=True,
                                    output_attentions=True)

#roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=True)
#roberta_model = RobertaModel.from_pretrained('roberta-base',
#                                    output_hidden_states=True,
#                                    output_attentions=True)


In [6]:
diff = find_difference(source, target, bert_tokenizer, bert_model)
print ('Similarity by BERT is ', diff)

#diff = find_difference(source, target, roberta_tokenizer, roberta_model)
#print ('Similarity by RoBERTa is ', diff)

Similarity by BERT is  [0.9245368]
